## Ideias para o sistema de recomendação

In [ ]:
## estudar esses imports

import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

A boa prática diz que você deve criar um range de tempo para analisar produtos, compras e interações que estavam disponíveis naquele período.
Isso faz sentido pois, se um dado produto foi ofertado depois e fez sucesso, ele pode não performar também ou não ter tantas interações quanto outro com mais tempo
o penalizando e fazendo com que o algoritmo não entenda a importância dessa combinação

Dar peso diferentes à interações de compra (Adicionar ao carrinho, prosseguir para pagamento, realizar pagamento)

Sistemas de recomendação sentem uma dificuldade em lidar com pouca informação, interação (user cold-start). Por isso, normalmente são separados para treinamento de modelo usuários com no mínimo um limiar de transações (Foi usado 5 no artigo base)

In [ ]:
### Como calcular essa quantidade de usuários com o limiar de transações 

users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

A avaliação do modelo pode ser feita por dois meios principais, pensando em cross-validation:
- Holdout - Deixando uma parte dos dados de interações compras de fora do modelo de treino para serem utilizados no modelo de teste. 
- Train test Split baseado em uma data - Definir uma data X para treinar em X-infinito e testar em X + infinito.

Acredito que para usar train teste temos que ter muitos dados de usuários, interações realmente muito recorrentes, como plataformas de streaming (música ou filmes/séries)
Em compras o melhor seria holdout, por não ter certeza que teremos tantas interações futuras assim

Nesse ponto entra também o problema "naive" que é crer que serão analisados somente usuários que estão presentes durante todo o recorte feito para treinamento e teste do modelo, o que sabemos que é praticamente impossível.

Medição de Acurácia nesses modelos

Em sistemas de recomendação, existem algumas métrias que são usadas de forma mais recorrente nas avaliação. Iremos utilizar métricas do tipo Top-N Accuracy Metrics, que avaliam a acurácia das recomendações principais à um usuário, comparando com os itens que ele realmente escolheu nos dados de teste.
Funciona da seguinte forma:

* Para cada usuário
    * Para cada item que esse usuário interagiu nos dados de teste
        * Crie uma amostra de outros 100 itens que esse usuário nunca interagiu   
        Ps. Aqui, novamente de forma "Naive" nós vamos assumir que os itens que o usuário nunca interagiu não são relevante para o mesmo, o que não pode ser verdade, pois ele pode não ter interagido com esses itens simplesmente por não saber da existência dos mesmos. Mas iremos manter assim
        * Peça para o modelo de recomendação produzir uma lista ranqueada dos 101 itens (o item que o usuário interagiu e os outros 100 "não relevantes")
        * Calcule e guarde a métrica de Top-N accuracy para esse usuário e o item que ele realmente interagiu na lista ranqueada produzida
* Junte todas as métricas do modelo

Métricas a serem utilizadas nesse tipo de avaliação (Top-N Accuracy Metrics):

- Recall@N - Iremos utilizar nesse modelo
- NDCG@N
- MAP@N

Link para mais informações sobre as outras duas [aqui](http://fastml.com/evaluating-recommender-systems/).

In [ ]:
# Indexando pelo ID para facilitar a separação dos grupos

interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])


## Parei aqui

#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator: